In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import ast
import xarray as xr
import matplotlib.pyplot as plt
from src.data_generator import *
from src.train import *

In [24]:
args = load_args('../nn_configs/B/42-resnet_cmip.yml')

In [27]:
args['train_years'] = [2015, 2015]

In [28]:
dg_train, dg_valid, dg_test = load_data(**args)

OSError: no files to open